<a href="https://colab.research.google.com/github/PatricioDieck/CheSS/blob/main/nanogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pprint

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-24 13:44:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-06-24 13:44:17 (18.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# read it to inspect the dataset
with open(r'input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset is", len(text), "characters")

length of dataset is 1115394 characters


In [ ]:
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("this vocab is", vocab_size, "characters long")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
this vocab is 65 characters long


In [ ]:
# develop strategy to tokenize text
# create mappings between characters and integers

stoi = { ch:i for i,ch in enumerate(chars)} #pairs str w/ ints in a dict
itos = { i:ch for i, ch in enumerate(chars)} #this is the same, just oposite (int:str pairs)

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])


print(encode('sup baby'))
print(decode(encode('sup baby')))

# lets mess around witht the enumerate() func for a sec

# print(list(enumerate('amongus')))
# print({ch:i for ch,i in enumerate('amongus')})
# print([i for ch,i in enumerate('amongus')])
# print([(ch,i) for ch,i in enumerate('amongus')])
# print({i:ch for i, ch in enumerate('hello world!')})


[57, 59, 54, 1, 40, 39, 40, 63]
sup baby


In [ ]:
# we're just fucking around with list and dict comprehensions here
import numpy as np
import pandas as pd

pd.array([ch for i, ch in enumerate('hello world!')])
pd.DataFrame([(i, ch) for i, ch in enumerate('hello world!')])

,0,1
0,0,h
1,1,e
2,2,l
3,3,l
4,4,o
5,5,
6,6,w
7,7,o
8,8,r
9,9,l


In [ ]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
# lets get a train/validation split of dataset

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(val_data[:100])

tensor([12,  0,  0, 19, 30, 17, 25, 21, 27, 10,  0, 19, 53, 53, 42,  1, 51, 53,
        56, 56, 53, 61,  6,  1, 52, 43, 47, 45, 46, 40, 53, 59, 56,  1, 14, 39,
        54, 58, 47, 57, 58, 39,  8,  0,  0, 14, 13, 28, 32, 21, 31, 32, 13, 10,
         0, 19, 53, 53, 42,  1, 51, 53, 56, 56, 53, 61,  6,  1, 52, 43, 47, 45,
        46, 40, 53, 59, 56,  1, 19, 56, 43, 51, 47, 53,  8,  0, 19, 53, 42,  1,
        57, 39, 60, 43,  1, 63, 53, 59,  6,  1])


In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size] #inputs to transformer
y = train_data[1:block_size+1] #targets- next character (offset by one from context)

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'when the input is {context}, the target is {target}')


when the input is tensor([18]), the target is 47
when the input is tensor([18, 47]), the target is 56
when the input is tensor([18, 47, 56]), the target is 57
when the input is tensor([18, 47, 56, 57]), the target is 58
when the input is tensor([18, 47, 56, 57, 58]), the target is 1
when the input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [ ]:
torch.manual_seed(1337)

batch_size = 4 #how many independent sequences will we process in parallel
block_size = 8 #maximum context length of prediciton?

def get_batch(split):
  # generate small barch of data of inputs x and targets y
  data = train_data if split == "train" else val_data #get appropriate dataset
  ix = torch.randint(len(data) - block_size, (batch_size,)) #get a rand number in dataset size

  x = torch.stack([data[i:i+block_size] for i in ix]) #get context for the block
  y = torch.stack([data[i+1:block_size+1] for i in ix]) #get targetfor each example

  return x, y


xb, yb = get_batch("train")

for b in range(batch_size): #batch dimensions\
  for t in range(block_size): #time dimension
    context= xb[b, :t+1]
    target= yb[b, t]
    print(f'when input is {context.tolist()} the target: {target}')

IndexError: index 0 is out of bounds for dimension 1 with size 0